# PaddleNLP词向量应用展示

6.7日NLP直播打卡课开始啦

**[直播链接请戳这里，每晚20:00-21:30👈](http://live.bilibili.com/21689802)**

**[课程地址请戳这里👈](https://aistudio.baidu.com/aistudio/course/introduce/24177)**

欢迎来课程**QQ群**（群号:618354318）交流吧~~


词向量（Word embedding），即把词语表示成实数向量。“好”的词向量能体现词语直接的相近关系。词向量已经被证明可以提高NLP任务的性能，例如语法分析和情感分析。


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/54878855b1df42f9ab50b280d76906b1e0175f280b0f4a2193a542c72634a9bf" width="60%" height="50%"> <br />
</p>
<br><center>图1：词向量示意图</center></br>

PaddleNLP已预置多个公开的预训练Embedding，您可以通过使用`paddlenlp.embeddings.TokenEmbedding`接口加载各种预训练Embedding。本篇教程将介绍`paddlenlp.embeddings.TokenEmbedding`的使用方法，计算词与词之间的语义距离，并结合词袋模型获取句子的语义表示。

token：模型输入基本单元。比如中文BERT中，token可以是一个字，也可以是<CLS>等标识符，一般情况下 token 是 word

embedding：一个用来表示token的稠密的向量。token本身不可计算，需要将其映射到一个连续向量空间，才可以进行后续运算，这个映射的结果就是该token对应的embedding。

## 加载TokenEmbedding

`TokenEmbedding()`参数
- `embedding_name`
将模型名称以参数形式传入TokenEmbedding，加载对应的模型。默认为`w2v.baidu_encyclopedia.target.word-word.dim300`的词向量。
- `unknown_token`
未知token的表示，默认为[UNK]。
- `unknown_token_vector`
未知token的向量表示，默认生成和embedding维数一致，数值均值为0的正态分布向量。
- `extended_vocab_path`
扩展词汇列表文件路径，词表格式为一行一个词。如引入扩展词汇列表，trainable=True。
- `trainable`
Embedding层是否可被训练。True表示Embedding可以更新参数，False为不可更新。默认为True。

In [ ]:
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

# 查看token_embedding详情
print(token_embedding)

### 认识一下Embedding
**`TokenEmbedding.search()`**
获得指定词汇的词向量。

In [ ]:
test_token_embedding = token_embedding.search("中国")
print(type(test_token_embedding))
import paddle
test_token_embedding_tensor = paddle.to_tensor(test_token_embedding)
print(test_token_embedding_tensor)

**`TokenEmbedding.cosine_sim()`**
计算词向量间余弦相似度，语义相近的词语余弦相似度更高，说明预训练好的词向量空间有很好的语义表示能力。

In [ ]:
score1 = token_embedding.cosine_sim("女孩", "女人")
score2 = token_embedding.cosine_sim("女孩", "书籍")
print('score1:', score1)
print('score2:', score2)

## 基于TokenEmbedding衡量句子语义相似度

在许多实际应用场景（如文档检索系统）中， 需要衡量两个句子的语义相似程度。此时我们可以使用词袋模型（Bag of Words，简称BoW）计算句子的语义向量。

**首先**，将两个句子分别进行切词，并在TokenEmbedding中查找相应的单词词向量（word embdding）。

**然后**，根据词袋模型，将句子的word embedding叠加作为句子向量（sentence embedding）。

**最后**，计算两个句子向量的余弦相似度。

### 基于TokenEmbedding的词袋模型


使用`BoWEncoder`搭建一个BoW模型用于计算句子语义。

* `paddlenlp.TokenEmbedding`组建word-embedding层
* `paddlenlp.seq2vec.BoWEncoder`组建句子建模层


In [ ]:
import paddle
import paddle.nn as nn
import paddlenlp


class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def get_cos_sim(self, text_a, text_b):
        text_a_embedding = self.forward(text_a)
        text_b_embedding = self.forward(text_b)
        cos_sim = self.cos_sim_func(text_a_embedding, text_b_embedding)
        return cos_sim

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)
        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)
        return summed

model = BoWModel(embedder=token_embedding)

### 构造Tokenizer
使用TokenEmbedding词表构造Tokenizer。

In [ ]:
from data import Tokenizer
tokenizer = Tokenizer()
print(token_embedding)

tokenizer.set_vocab(vocab=token_embedding.vocab)

### 相似句对数据读取


In [ ]:
text_pairs = {}
with open("text_pair.txt", "r", encoding="utf8") as f:
    for line in f:
        text_a, text_b = line.strip().split("\t")
        if text_a not in text_pairs:
            text_pairs[text_a] = []
        text_pairs[text_a].append(text_b)

### 查看相似语句相关度

In [ ]:
for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])
    embedded_text = token_embedding(text_a_ids)

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        print("text_a: {}".format(text_a))
        print("text_b: {}".format(text_b))
        print("cosine_sim: {}".format(model.get_cos_sim(text_a_ids, text_b_ids).numpy()[0]))
        print()
        break
